# Ajax-爬取微博博主的所有微博数据

首先导入必要的包

In [13]:
import requests
import json
from urllib.parse import urlencode
import re
import time
from pyquery import PyQuery as pq

设置基准网址以及headers

In [3]:
base_url = 'https://m.weibo.cn/api/container/getIndex?'
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'Host':'m.weibo.cn',
    'Referer':'https://m.weibo.cn/u/1752054291',
    'X-Requested-With': 'XMLHttpRequest'
}

设置参数测试一下返回结果，ok！

In [4]:
params = {
    'type': 'uid',
    'value': '1752054291',
    'containerid': '1076031752054291',
    'page':1
}
url = base_url + urlencode(params)
r = requests.get(url,headers)
r

<Response [200]>

**定义获取JSON数据的函数**

In [5]:
def get_data(index):
    params = {
    'type': 'uid',
    'value': '1752054291',
    'containerid': '1076031752054291',
    'page':index
    }
    url = base_url + urlencode(params)
    try:
        r = requests.get(url,headers)
        if r.status_code == 200:
            return r.json()
    except requests.ConnectionError as e:
        print("error",e.args)

定义处理JSON数据并且返回迭代器的函数

In [10]:
def parse_data(j_data):
    if j_data:
        its = j_data.get('data').get('cards')
        for it in its:
            if it.get('card_type') != 9:
                continue
            it = it.get('mblog')
            weibo = {}
            weibo['id'] = it.get('id')
            weibo['text'] = pq(it.get('text')).text()
            weibo['reposts'] = it.get('reposts_count')
            weibo['comments'] = it.get('comments_count')
            weibo['attitudes'] = it.get('attitudes_count')
            weibo['pic'] = it.get('original_pic')
            yield weibo

定义写文件的函数

In [9]:
def write_data(weibos,index):
    with open("weibo.md",'a+',encoding='utf-8') as f:
        f.write('# 第%d页\n' % index)
        for x in weibos:
            f.write(x['text']+'\n')
            f.write('点赞数:'+str(x['attitudes'])+' 转发数:'+str(x['reposts'])+ ' 评论数:'+str(x['comments'])+'\n')
            f.write('![](%s)\n\n' % x['pic'])
        print('page %d write ok!' % index)

设置爬取的最新微博的页数并且开始爬取

In [15]:
page_num = 10
for index in range(1,page_num):
    j_data = get_data(index)
    result = parse_data(j_data)
    write_data(result,index)
    time.sleep(2)

page 1 write ok!
page 2 write ok!
page 3 write ok!
page 4 write ok!
page 5 write ok!
page 6 write ok!
page 7 write ok!
page 8 write ok!
page 9 write ok!
